In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

In [ ]:
exp_types = ['lime']
train_sizes = [4, 16, 64, 100, 1000]
random_seeds = [0, 1, 2]
val_size = 250
weight_decay = 1e-4
lr = 1e-4
epochs = 1000
batchsize = 16

## Define agent model

First, need to define agent architecture. We show one possible architecture (the DeepSet model).

In [ ]:
class DeepSet(nn.Module):

    def __init__(self, set_size, in_features, set_features=100): #50
        super(DeepSet, self).__init__()
        self.in_features = in_features
        self.out_features = set_features
        self.feature_extractor = nn.Sequential(
            nn.Linear(in_features, 200), #100
            nn.ELU(inplace=True),
            nn.Linear(200, 100), #100
            nn.ELU(inplace=True),
            nn.Linear(100, set_features)
        )

        self.regressor = nn.Sequential(
            nn.Linear(set_features, 30),
            nn.ELU(inplace=True),
            nn.Linear(30, 30),
            nn.ELU(inplace=True),
            nn.Linear(30, 10),
            nn.ELU(inplace=True),
            nn.Linear(10, 1),
            nn.Sigmoid()
        )

        self.weighted_sum = nn.Conv1d(in_channels=set_size, out_channels=1, kernel_size=1)
        
        self.add_module('0', self.feature_extractor)
        self.add_module('1', self.regressor)
        
        
    def reset_parameters(self):
        for module in self.children():
            reset_op = getattr(module, "reset_parameters", None)
            if callable(reset_op):
                reset_op()
            
    def forward(self, input):
        x = input
        x = self.feature_extractor(x)
        x = x.sum(dim=1) 
        x = self.regressor(x)
        return x

    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'Feature Exctractor=' + str(self.feature_extractor) \
            + '\n Set Feature' + str(self.regressor) + ')'


In [ ]:
class DatasetDeepset(Dataset):
    def __init__(self, n, X, y, input_dim: int):
        self.n = n
        #load files
        self.X = X
        self.y = y

        self.input_dim = input_dim

    def __getitem__(self, index):
        item = self.X[index,:]
        item = item.reshape((1, self.input_dim))
        return torch.from_numpy(item).type(torch.FloatTensor), torch.from_numpy(np.array([self.y[index]])).type(torch.FloatTensor)

    def __len__(self):
        return self.n

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("using device", device)

## Train agent model

In [ ]:
def data_by_ind(ind):
    X = d['x'][ind]
    y = d['y'][ind].flatten()
    u = d['u'][ind].flatten()
    lime = d['lime'][ind]
    
    return X, y, u, lime

def load_train_and_test_data(N_train, N_test, ind):
      
    train_ind_subset = ind[:N_train]
    test_ind_subset = ind[-N_test:]
        
    X_train, y_train, u_train, lime_train = data_by_ind(train_ind_subset)
    
    X_test, y_test, u_test, lime_test = data_by_ind(test_ind_subset)
        
    return X_train, y_train, u_train, lime_train, X_test, y_test, u_test, lime_test
        

In [ ]:
for seed in random_seeds:
    print(f'seed {seed}')
    
    # default rs 0: select the first N/2 points from each dataframe
    ind = np.arange(original_len(d))
    
    if seed != 0:
        # randomly shuffle the order in which we select from the dataframes
        np.random.shuffle(ind)

    for train_size in train_sizes:
        print(f'size {train_size}')
        X_train, y_train, u_train, lime_train, shap_train, gam_train, X_test, y_test, u_test, lime_test, shap_test, gam_test = load_train_and_test_data(N_train = train_size, N_test = 250, ind = ind)
        
        for exp_str in exp_types:
            # only use a train set of the given size
            exp_train = None
            exp_test = None
            if exp_str == 'lime':
                exp_train = lime_train
                exp_test = lime_test

            train_loader, train_dim, X_train_tensor, u_train_tensor = get_loader(X_train, u_train, 
                                     exp = exp_train, 
                                     batchsize = batchsize)

            test_loader, test_dim, X_test_tensor, u_test_tensor = get_loader(X_test, u_test, 
                                           exp = exp_test,
                                           batchsize = batchsize)
            
            model = DeepSet(set_size = 1, 
                in_features = train_dim)

            model.to(device=device)
            
            model.train_accs = []
            model.test_accs = []

            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
            
            # Defining Loss Function
            criterion = nn.BCELoss()

            # Define Training Loop
            def train(epoch, loss_list):
                model.train()
                for batch_idx, (image, mask) in enumerate(train_loader):
                    image = image.to(device=device)
                    mask = mask.to(device=device)
                    mask = mask.squeeze(1)
                    image, mask = Variable(image), Variable(mask)

                    optimizer.zero_grad()

                    output = model(image)

                    loss = criterion(output, mask.unsqueeze(1)) 
                    loss_list.append(loss.item())

                    loss.backward()
                    optimizer.step()

            def evaluate_accuracy(X_tensor, u_true_tensor, is_train):
                u_pred_soft = model(X_tensor)
                u_pred_hard = np.round(u_pred_soft.detach()).flatten()
                acc = accuracy_score(u_true_tensor.detach().flatten(), u_pred_hard)
                
                if is_train:
                    model.train_accs.append(acc)
                else:
                    model.test_accs.append(acc)


            loss_list = []
            for i in range(epochs):
                train(i, loss_list)
                evaluate_accuracy(X_train_tensor, u_train_tensor, is_train = True)
                evaluate_accuracy(X_test_tensor, u_test_tensor, is_train = False)


            # store trained model
            torch.save(model.state_dict(),
                       f'models/{version_substr}_rs{seed}_wd={weight_decay}_model_trainsize_{train_size}_exptype_{exp_str}.pt')

            # dump validation accuracies and train accuracies
            pickle.dump(model.train_accs, open(f'train_accs.pt', "wb" ) )
            pickle.dump(model.test_accs, open(f'test_accs.pt', "wb" ) )